<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">Classification with Python</font></h1>

In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset

This dataset is about past loans. The __Loan_train.csv__ data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |

Lets download the dataset

In [ ]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

### Load Data From CSV File  

In [ ]:
df = pd.read_csv('loan_train.csv')
df.head()

In [ ]:
df.shape

### Convert to date time object 

In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

# Data visualization and pre-processing



Let’s see how many of each class is in our data set 

In [ ]:
df['loan_status'].value_counts()

260 people have paid off the loan on time while 86 have gone into collection 


Lets plot some columns to underestand data better:

In [ ]:
# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction

### Lets look at the day of the week people get the loan 

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that people who get the loan at the end of the week dont pay it off, so lets use Feature binarization to set a threshold values less then day 4 

In [ ]:
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

## Convert Categorical features to numerical values

Lets look at gender:

In [ ]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

86 % of female pay there loans while only 73 % of males pay there loan


Lets convert male to 0 and female to 1:


In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

## One Hot Encoding  
#### How about education?

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)

#### Feature befor One Hot Encoding

In [ ]:
df[['Principal','terms','age','Gender','education']].head()

#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame 

In [ ]:
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()


### Feature selection

Lets defind feature sets, X:

In [ ]:
X = Feature
X[0:5]

What are our lables?

In [ ]:
y = df['loan_status'].values
y[0:5]

## Normalize Data 

Data Standardization give data zero mean and unit variance (technically should be done after train test split )

In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

# Classification 

Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:
- K Nearest Neighbor(KNN)
- Decision Tree
- Support Vector Machine
- Logistic Regression



__ Notice:__ 
- You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
- You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
- You should include the code of the algorithm in the following cells.

# K Nearest Neighbor(KNN)
Notice: You should find the best k to build the model with the best accuracy.  
**warning:** You should not use the __loan_test.csv__ for finding the best k, however, you can split your train_loan.csv into train and test to find the best __k__.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# Iterate to find the k-value
ks = 60
mean_acc = np.zeros((ks - 1))
std_acc = np.zeros((ks - 1))
for n in range(1, ks):
    # Train Model and Predict
    neigh = KNeighborsClassifier(n_neighbors=n).fit(x_train, y_train)
    yhat = neigh.predict(x_test)
    mean_acc[n - 1] = metrics.accuracy_score(y_test, yhat)
    std_acc[n - 1] = np.std(yhat == y_test) / np.sqrt(yhat.shape[0])

plt.plot(range(1, ks), mean_acc, 'g')
plt.fill_between(range(1, ks), mean_acc - 1 * std_acc, mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 1xstd', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Neighbors (K)')
plt.tight_layout()
plt.show()

In [ ]:
print("The best accuracy was with ", mean_acc.max(), "with k=", mean_acc.argmax() + 1)

# Decision Tree

In [ ]:
''' Decision Tree '''
from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier(criterion="entropy", max_depth=4)
model_tree.fit(x_train, y_train)
yhat = model_tree.predict(x_test)

In [ ]:
from sklearn import metrics
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, yhat))

In [ ]:
# Notice: You might need to uncomment and install the pydotplus and graphviz libraries if you have not installed these before
# !conda install -c conda-forge pydotplus -y
# !conda install -c conda-forge python-graphviz -y
try:
    from io import StringIO
    import pydotplus
    import matplotlib.image as mpimg
    from sklearn import tree
    dot_data = StringIO()
    filename = "result_tree.png"
    out = tree.export_graphviz(model_tree, feature_names=None,
                               out_file=dot_data, class_names=np.unique(y_test),
                               filled=True,  special_characters=True, rotate=False)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    graph.write_png(filename)
    img = mpimg.imread(filename)
    plt.figure(figsize=(100, 200))
    plt.imshow(img, interpolation='nearest')
except ModuleNotFoundError as ex:
    print(" >>> ModuleNotFoundError :")
    print(ex)

# Support Vector Machine

In [ ]:
from sklearn import svm
model_svn = svm.SVC(kernel='rbf')
model_svn.fit(x_train, y_train)

In [ ]:
yhat = clf.predict(x_test)
print(yhat[0:5])

In [ ]:
print("SVM's Accuracy: ", metrics.accuracy_score(y_test, yhat))

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(C=0.01, solver='liblinear').fit(x_train, y_train)

In [ ]:
yhat = model_lr.predict(x_test)
print(yhat[0:5])

In [ ]:
print("LR's Accuracy: ", metrics.accuracy_score(y_test, yhat))

# Model Evaluation using Test set

In [ ]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

First, download and load the test set:

In [ ]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

### Load Test set for evaluation 

In [ ]:
test_df = pd.read_csv('loan_test.csv')
test_df.head()

In [ ]:
#  Normalyze as train df
df_test['due_date'] = pd.to_datetime(df['due_date'])
df_test['effective_date'] = pd.to_datetime(df['effective_date'])
df_test['dayofweek'] = df['effective_date'].dt.dayofweek
df_test['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x > 3) else 0)
df_test['Gender'].replace(to_replace=['male', 'female'], value=[0, 1], inplace=True)
print(df_test.head())

# Use one hot encoding technique to conver categorical varables to binary variables
# and append them to the feature DataFrame
Feature_test = df_test[['Principal', 'terms', 'age', 'Gender', 'weekend']]
Feature_test = pd.concat([Feature_test, pd.get_dummies(df_test['education'])], axis=1)
Feature_test.drop(['Master or Above'], axis=1, inplace=True)
print(Feature_test.head())

# Feature selection, lets defind feature sets, X:
xset_test = Feature_test
print(xset_test[0:5])

# Data Standardization give data zero mean and unit variance (technically should be done after train test split )
xset_test = preprocessing.StandardScaler().fit(xset_test).transform(xset_test)
print(xset_test[0:5])

In [ ]:
# What are our labels?
yset_test = df_test['loan_status'].values
print(yset_test[0:5])

Predictions for our test train

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=7).fit(x_train, y_train)  # we found previously 7 as best k-value
yhat_knn = model_knn.predict(xset_test)
yhat_tree = model_tree.predict(xset_test)
yhat_svm = model_svn.predict(xset_test)
yhat_lr = model_lr.predict(xset_test)

F1 scores :

In [ ]:
from sklearn.metrics import f1_score
print("f1_score knn: : %.2f" % f1_score(yset_test, yhat_knn, average='weighted'))
print("f1_score tree: : %.2f" % f1_score(yset_test, yhat_tree, average='weighted'))
print("f1_score svm: : %.2f" % f1_score(yset_test, yhat_svm, average='weighted'))
print("f1_score lr: : %.2f" % f1_score(yset_test, yhat_lr, average='weighted'))

Jaccard similarity coefficient :

In [ ]:
from sklearn.metrics import jaccard_score
print("jaccard_score knn: : %.2f" % jaccard_score(yset_test, yhat_knn, average='weighted'))
print("jaccard_score tree: : %.2f" % jaccard_score(yset_test, yhat_tree, average='weighted'))
print("jaccard_score svm: : %.2f" % jaccard_score(yset_test, yhat_svm, average='weighted'))
print("jaccard_score lr: : %.2f" % jaccard_score(yset_test, yhat_lr, average='weighted'))

Log loss, aka logistic loss or cross-entropy loss.

In [ ]:
yhat_lr_proba = model_lr.predict_proba(xset_test)
from sklearn.metrics import log_loss
print("LogLoss: : %.2f" % log_loss(yset_test, yhat_lr_proba))

# Report
You should be able to report the accuracy of the built model using different evaluation metrics:

| Algorithm          | Jaccard | F1-score | LogLoss |
|--------------------|---------|----------|---------|
| KNN                | 0.53    | 0.65     | NA      |
| Decision Tree      | 0.37    | 0.54     | NA      |
| SVM                | 0.68    | 0.79     | NA      |
| LogisticRegression | 0.55    | 0.63     | 0.60    |

<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

<p>Copyright &copy; 2018 <a href="https://cocl.us/DX0108EN_CC">Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.</p>